# SEP532 인공지능 이론과 실제
## Deep Learning Practice 
#### Prof. Ho-Jin Choi
#### School of Computing, KAIST

---

### 4-3. Feature extraction using pre-trained model

Constructing and training your own CNN models from scratch can be hard and a long task. A common trick used in deep learning is to use a **pre-trained** model and **fine-tune** it to the specific data it will be used for. 

The pre-trained model is a saved network that was previously trained on a large dataset, typically on a large-scale image-classificaton task. For the pre-trained model, we can either **use the pre-trained model as it is**, or **use transfer learning to customize this model** to a given task. 

The **intuition behind transfer learning** is that if a model trained on a large and general enough dataset, this model will effectively serve as a generic model of the visual world. We can then take advantage of these learned feature maps without having to start from scratch training a large model on a large dataset.

In this practice, we will try two ways to customize a pre-trained model:
1. **Feature Extraction**: To extract meaningful features from new samples. We **simply add a new classifier**, which will be trained from scratch, **on top of the pretrained model** so that we can repurpose the feature maps learned previously for our dataset. To classify our task, we will **train the final parts of model**, classification part of the pre-trained model, thus, we do not need to (re)train the entire model.
1. **Fine-tuning**: Unfreezing a few of the top layers of a frozen model base and **jointly training both the newly-added classifier layers and the last layers of the base model**. This allows us to "fine tune" the higher-order feature representations in the base model in order to make them more relevant for the specific task.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import os
import numpy as np
import matplotlib.pyplot as plt

#### Data preprocessing

We will first download and load the cats and dogs dataset using the [`Tensorflow Datasets`](https://www.tensorflow.org/datasets).
This [`tfds`](https://www.tensorflow.org/datasets/api_docs/python/tfds) package is the easiest way to load pre-defined data.

(If you want to use your own data and are interested in importing it, see the TensorFlow loading image data, [`tf.data`](https://www.tensorflow.org/beta/tutorials/load_data/images).

In [ ]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

The [`tfds.load`](https://www.tensorflow.org/datasets/api_docs/python/tfds/load) method downloads and caches the data, and returns a [`tf.data.Dataset`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/data/Dataset) object. These objects provide powerful, efficient methods for manipulating data and piping it into your model.

Since "cats_vs_dog" doesn't define standard splits, use the subsplit feature to divide it into (train, validation, test) with 80%, 10%, 10% of the data respectively.

([`tfds.Split.TRAIN.subsplit`](https://www.tensorflow.org/datasets/api_docs/python/tfds/core/SplitBase?hl=en#subsplit): Divides this split into subsplits.)

In [ ]:
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

The resulting [`tf.data.Dataset`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/data/Dataset) objects contain **(image, label) pairs**. Where the images have variable shape and 3 channels, and the label is a scalar.

In [ ]:
print(raw_train)
print(raw_validation)
print(raw_test)

Show the first two images and labels from the training set:
([`tfds.features.ClassLabel.int2str`](https://www.tensorflow.org/datasets/api_docs/python/tfds/features/ClassLabel?hl=en#int2str): Conversion integer => class name string.)

In [ ]:
get_label_name = metadata.features['label'].int2str

for image, label in raw_train.take(2):
    plt.figure()
    plt.imshow(image)
    plt.title(get_label_name(label))

#### Format the data

Use the [`tf.image`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/image) module to format the images for the task.

Resize the images to a fixes input size, and rescale the input channels to a range of [-1,1]

In [ ]:
IMG_SIZE = 160 # All images will be resized to 160x160

def format_example(image, label):
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

[`tf.data.Dataset.map`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/data/Dataset#map)(map_func, num_parallel_calls=None)

Maps `map_func` across the elements of this dataset.

This transformation applies map_func to each element of this dataset, and returns a new dataset containing the transformed elements, in the same order as they appeared in the input.

Apply this function to each item in the dataset using the map method:

In [ ]:
# There is map function in tf.
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

[`tf.data.Dataset.shuffle`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/data/Dataset#shuffle)(buffer_size, seed=None, reshuffle_each_iteration=None)

Randomly shuffles the elements of this dataset.

[`tf.data.Dataset.batch`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/data/Dataset#batch)(batch_size, drop_remainder=False)

Combines consecutive elements of this dataset into batches.

Now shuffle and batch the data.

In [ ]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

In [ ]:
# after batch function, (BATCH_SIZE, image_width, image_height, channel)


Inspect a batch of data:

In [ ]:
for image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

#### Create the base model from the pre-trained convnets

We will create the base model from the **MobileNet V2 model** developed at Google. This is pre-trained on the ImageNet dataset, a large dataset of 1.4M images and 1000 classes of web images. 

First, we need to **pick which layer of MobileNet V2 we will use for feature extraction**. Obviously, the very last classification layer is not very useful. Instead, we will follow the common practice to instead depend on the **very last layer before the flatten operation**. This layer is called the "_bottleneck layer_". The **bottleneck features retain much generality** as compared to the final/top layer.

To do this, instantiate a MobileNet V2 model pre-loaded with weights trained on ImageNet. By specifying the _include_top=False_ argument, we can **load a network that doesn't include the classification layers** at the top, which is ideal for feature extraction.

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = 

This feature extractor **converts each 160x160x3 image to a 5x5x1280 block of features**. See what it does to the example batch of images:

In [ ]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)

#### Freeze the convolutional base for feature extraction

We will **freeze the convolutional base** created from the previous step and **use that as a feature extractor**, **add a classifier on top of it** and **train the top-level classifier**.

It's important to freeze the convolutional based before we compile and train the model. By freezing (or setting _layer.trainable = False_), we prevent the weights in a given layer from being updated during training. MobileNet V2 has many layers, so **setting the entire model's trainable flag to _False_ will freeze all the layers**.

In [ ]:
# Set the trainable flag to 'False' for all of the conv and pooling layers of base model 


In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

#### Add a classification head

To generate predictions from the block of features, average over the spatial 5x5 spatial locations, using a [`tf.keras.layers.GlobalAveragePooling2D`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/GlobalAveragePooling2D) layer to convert the features to a single 1280-element vector per image.

<img src=https://alexisbcook.github.io/assets/global_average_pooling.png width="500">


In [ ]:
# (32, 5, 5, 1280) -> (32, 1280)
# extract scalar value for each 5x5 feature by applying average pooling 

print(feature_batch_average.shape)

Apply a `tf.keras.layers.Dense` layer to convert these features into a single prediction per image. We don't need an activation function here because this prediction will be treated as a logit, or a raw prediction value. Positive numbers predict class 1, negative numbers predict class 0.

In [ ]:
# cheack prediction batch shape by adding one Dense layer 

print(prediction_batch.shape)

Now stack the feature extractor, and these two layers using a `tf.keras.Sequential` model:

In [ ]:
# define the model 
model = tf.keras.Sequential([
])

#### Compile the model
You must compile the model before training it. Since there are two classes, use a binary cross-entropy loss.

In [ ]:
# optimizer: RMSprop
# loss: binary crossentropy
# metrics: accuracy

base_learning_rate = 0.0001
model.compile()

model.summary()

The 2.5M parameters in MobileNet are frozen, but there are 1.2K trainable parameters in the Dense layer. These are divided between two `tf.Variable` objects, the weights and biases.

In [ ]:
len(model.trainable_variables)

#### Train the model

In [ ]:
initial_epochs = 10
validation_steps=20

loss0, accuracy0 = 

In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

The accuracy is same to randomly select the answer. So, we will check an accuracy after training the model.

In [ ]:
history = model.fit(

)

#### Learning curves

Let's take a look at the learning curves of the training and validation accuracy/loss when using the MobileNet V2 base model as a fixed feature extractor.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

- 그래프에서 validation이 loss 더 빨리 감소하는 이유: training하는 동안 validation data를 이용하여 validation loss를 구할떄 batch normalization이나 dropout이 사용되지 않기 때문임

- validation loss 그래프가 training loss 그래프보다 긴 이유: training loss는 각 epoch에 대한 평균이 그려짐, 반면 validation loss그래프는 epoch 끝난후 바로바로 evaluation 되기 때문

### 4-4. Fine tuning our classification model 

In the previous feature extraction part, we were only training a few layers on top of an MobileNet V2 base model. The weights of the pre-trained network were not updated during training.

**One way to increase performance** even further is to **train (or "fine-tune") the weights of the top layers of the pre-trained model** alongside the training of the classifier you added. The training process will force the **weights to be tuned from generic features maps to features associated specifically to our dataset**.

Also, we should try to fine-tune a **small number of top layers** rather than the whole MobileNet model. In most convolutional networks, **the higher up a layer is, the more specialized it is**. The first few layers learn very simple and generic features which generalize to almost all types of images. As we go higher up, the features are increasingly more specific to the dataset on which the model was trained. **The goal of fine-tuning is to adapt these specialized features to work with the new dataset, rather than overwrite the generic learning.**

#### Un-freeze the top layers of the model
All we need to do is **unfreeze the base_model** and **set the bottom layers be un-trainable**.(i.e., we will fine-tune a small number of top layers of base mode). Then, you should recompile the model (necessary for these changes to take effect), and resume training.


In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine tune from this layer onwards
fine_tune_at = 

# Freeze all the layers before the `fine_tune_at` layer


#### Compile the model
Compile the model using a much lower training rate.

(Our model:  <br>
&emsp;&emsp;base_model, <br>
&emsp;&emsp;global_average_layer,  <br>
&emsp;&emsp;prediction_layer)

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

#### Continue Train the model
If you trained to convergence earlier, this will get you a few percent more accuracy.

In [ ]:
fine_tune_epochs = 3
total_epochs =  initial_epochs + fine_tune_epochs


# initial_epoch: Integer. Epoch at which to start training (useful for resuming a previous training run).
history_fine = model.fit(

)

Let's take a look at the learning curves of the training and validation accuracy/loss, when fine tuning the last few layers of the MobileNet V2 base model and training the classifier on top of it. The validation loss is much higher than the training loss, so you may get some overfitting.

You may also get some overfitting as the new training set is relatively small and similar to the original MobileNet V2 datasets.

**After fine tuning the model nearly reaches 98% accuracy.**

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()